# Installing Packages

By running the command **`pip install BeautifulSoup4`** in your terminal or notebook

In [ ]:
!pip install selenium

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 460.2/460.2 kB 38.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.4 MB/s eta 0:00:00


# Importing necessary dependencies

In [ ]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import time
from sqlalchemy import create_engine

# Test for response 200

In [ ]:
url = "https://www.aliexpress.com/w/wholesale-laptop.html?g=y&SearchText=laptop"

r = requests.get(url)

#print(r.status_code)
print(r)

<Response [200]>


In [ ]:
# Configure Selenium ChromeDriver options
options = Options()
options.headless = True  # Run in headless mode (without opening a browser window)
service = Service(executable_path=r'C:\Users\princ\Downloads\Ehi_Ughele_Travel_documents\10nalytics\mastterclass\Aliexpress_pipeline\chromedriver-win64\chromedriver-win64\chromedriver.exe')  # Update this path

# Initialize the WebDriver
driver = webdriver.Chrome(service=service, options=options)

# Define the URL
url = "https://www.aliexpress.com/w/wholesale-laptops.html?spm=a2g0o.productlist.search.0"

# Use Selenium to open the page
driver.get(url)

# Wait for the dynamic content to load
time.sleep(10)  # Adjust the sleep time as needed

# Get the page source and close the browser
page_source = driver.page_source
driver.quit()


# Extraction Layer

In [ ]:
# Configure Selenium ChromeDriver options
options = Options()
options.headless = True  # Run in headless mode (without opening a browser window)
service = Service(executable_path=r'C:\Users\princ\Downloads\Ehi_Ughele_Travel_documents\10nalytics\mastterclass\Aliexpress_pipeline\chromedriver-win64\chromedriver-win64\chromedriver.exe')  # Update this path

# Initialize the WebDriver
driver = webdriver.Chrome(service=service, options=options)

# Lists to store extracted data
product_names = []
prices = []
store_links = []
shipping_prices = []
extra_discounts = []
items_sold = []
sales_prices = []
original_prices = []
discount_percentages = []
shipping_free_statuses = []

# Assuming you know the total number of pages or finding it dynamically could be complex
total_pages = 1  # This could be adjusted based on the site's navigation options

for page_number in range(1, total_pages + 1):
    url = f"https://www.aliexpress.com/w/wholesale-laptop.html?page={page_number}&g=y&SearchText=laptop"
    driver.get(url)
    time.sleep(10)  # Adjust the sleep time as needed for the page to fully load

    # Now use BeautifulSoup to parse the loaded page source
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    # Adjust the class selectors based on the current website structure
    laptops = soup.find_all('div', class_='list--gallery--C2f2tvm search-item-card-wrapper-gallery')  # Update class based on current website structure

    for laptop in laptops:
        # The rest of your scraping logic here, make sure to update class names as necessary
        # Example for product name, update class names as per the website's current structure
        product_name = laptop.find('div', class_='multi--title--G7dOCj3').text
        price = laptop.find('div', class_='multi--price-sale--U-S0jtj').text.replace('NGN', '').replace(',', '')

        try:
            store_link = laptop.find('a', class_='cards--storeLink--XkKUQFS')['href']
        except (TypeError, KeyError):
            store_link = ''

        try:
            shipping_price = laptop.find('span', class_='tag--text--1BSEXVh 2').text
        except AttributeError:
            shipping_price = ''

        try:
            extra_discount = laptop.find('span', class_='tag--text--1BSEXVh').text
        except AttributeError:
            extra_discount = ''

        try:
            items_sold_text = laptop.find('span', class_='multi--trade--Ktbl2jB').text
            items_sold_value = items_sold_text.split()[0] if items_sold_text else ''
        except AttributeError:
            items_sold_value = ''

        # Append data to lists
        product_names.append(product_name)
        prices.append(price)
        store_links.append(store_link)
        shipping_prices.append(shipping_price)
        extra_discounts.append(extra_discount)
        items_sold.append(items_sold_value)

        # Extract additional information
        new_price = laptop.find_all("div", class_="multi--price-sale--U-S0jtj")
        sales_prices.append(new_price[0].text if new_price else '')

        former_price = laptop.find_all("div", class_="multi--price-original--1zEQqOK")
        original_prices.append(former_price[0].text if former_price else '')

        discount = laptop.find_all("div", class_="multi--dealContainer--T8SnTim")
        discount_perc = discount[0].text.strip() if discount else ''
        discount_percentages.append(discount_perc)

        free_shipping = laptop.find_all("div", class_="multi--serviceContainer--3vRdzWN")
        ship_free = free_shipping[0].text.strip() if free_shipping else ''
        shipping_free_statuses.append(ship_free)


# Once all pages are processed, quit the driver
driver.quit()

# Create a DataFrame
data = {
    'Product Name': product_names,
    'Price': prices,
    'Store Link': store_links,
    'Shipping Price': shipping_prices,
    'Extra Discount': extra_discounts,
    'Items Sold': items_sold,
    'Sales Price': sales_prices,
    'Original Price': original_prices,
    'Discount Percentage': discount_percentages,
    'Shipping Free Status': shipping_free_statuses
}

df = pd.DataFrame(data)

# Display the DataFrame
print(df)

                                        Product Name       Price  \
0  CHUWI MiniBook X Laptop Tablet 2 In 1 Intel N1...   618885.94   
1  MSI Titan 18 Ultra 2024 14th Generation 18-inc...  9355525.54   
2  FIREBAT A14 Laptop Intel N5095 14.1 Inch 16GB ...   421269.19   
3  ASUS ROG Strix SCAR 18 Gaming Laptop Intel Cor...     5611035   
4  VCHANCE 15.6 Inch 1080p Portable Monitor Built...   300975.91   
5  FIREBAT T6A-X 16 Inch Intel i9-12900HX RTX 406...  1945345.84   
6  11th Gen i7 Gaming Laptop MX450 Graphics 15.6 ...  1017596.28   
7  2023 New 11th Gen 11.6-inch Thin Lightweight L...   232982.41   
8  ASUS Gaming Laptop ROG Strix SCAR 18 Intel Cor...    11222070   
9  VCHANCE 15 Inch Touch Screen Portable Monitor ...   228593.57   

                              Store Link Shipping Price  \
0  //www.aliexpress.com/store/1102773765                  
1  //www.aliexpress.com/store/1102964053                  
2     //www.aliexpress.com/store/5250057                  
3  //www.aliexp

In [ ]:
df.head(30)

,Product Name,Price,Store Link,Shipping Price,Extra Discount,Items Sold,Sales Price,Original Price,Discount Percentage,Shipping Free Status
0,CHUWI MiniBook X Laptop Tablet 2 In 1 Intel N1...,618885.94,//www.aliexpress.com/store/1102773765,,Extra 2% off with coins,333,"NGN618,885.94","NGN1,785,301.16",Extra 2% off with coins-65%,Free shipping
1,MSI Titan 18 Ultra 2024 14th Generation 18-inc...,9355525.54,//www.aliexpress.com/store/1102964053,,Extra 2% off with coins,1,"NGN9,355,525.54",,Extra 2% off with coins,"+Shipping: NGN120,554.21"
2,FIREBAT A14 Laptop Intel N5095 14.1 Inch 16GB ...,421269.19,//www.aliexpress.com/store/5250057,,Extra 2% off with coins,97,"NGN421,269.19","NGN953,988.48",Extra 2% off with coins-55%,Free shipping
3,ASUS ROG Strix SCAR 18 Gaming Laptop Intel Cor...,5611035,//www.aliexpress.com/store/1102248573,,"+Shipping: NGN381,105.99",1,"NGN5,611,035",,,"+Shipping: NGN381,105.99"
4,VCHANCE 15.6 Inch 1080p Portable Monitor Built...,300975.91,//www.aliexpress.com/store/1102820440,,Extra 5% off with coins,8,"NGN300,975.91",,Extra 5% off with coins,Free shipping
5,FIREBAT T6A-X 16 Inch Intel i9-12900HX RTX 406...,1945345.84,//www.aliexpress.com/store/1102531135,,"+Shipping: NGN399,907.44",103,"NGN1,945,345.84",,,"+Shipping: NGN399,907.44"
6,11th Gen i7 Gaming Laptop MX450 Graphics 15.6 ...,1017596.28,//www.aliexpress.com/store/5000334,,"+Shipping: NGN242,566.93",1,"NGN1,017,596.28",,,"+Shipping: NGN242,566.93"
7,2023 New 11th Gen 11.6-inch Thin Lightweight L...,232982.41,//www.aliexpress.com/store/1103208214,,Extra 2% off with coins,2,"NGN232,982.41","NGN568,263.88",Extra 2% off with coins-59%,Free shipping
8,ASUS Gaming Laptop ROG Strix SCAR 18 Intel Cor...,11222070,//www.aliexpress.com/store/1102252612,,"+Shipping: NGN381,105.99",,"NGN11,222,070",,,"+Shipping: NGN381,105.99"
9,VCHANCE 15 Inch Touch Screen Portable Monitor ...,228593.57,//www.aliexpress.com/store/1102820440,,Extra 5% off with coins,1,"NGN228,593.57",,Extra 5% off with coins,Free shipping


In [ ]:
# Save the raw data
df.to_csv("AliExpressLaptops.csv", index=True)

### Transaction Layer

In [ ]:
# To see the columns
df.columns

Index(['Product Name', 'Price', 'Store Link', 'Shipping Price',
       'Extra Discount', 'Items Sold', 'Sales Price', 'Original Price',
       'Discount Percentage', 'Shipping Free Status'],
      dtype='object')

In [ ]:
# To view data schema info
df.info()

In [ ]:
# Load data from CSV file
csv_file_path = 'AliExpressLaptops.csv'
df = pd.read_csv(csv_file_path)

In [ ]:
# Product Table
product_columns = ['Product Name', 'Price', 'Store Link']
products = df[product_columns].copy()

# Add ProductID column with unique values
products['ProductID'] = range(1, len(products) + 1)

# Reorder columns for consistency with the ERD schema
products = products[['ProductID', 'Product Name', 'Price', 'Store Link']]


In [ ]:
products.head()

,ProductID,Product Name,Price,Store Link
0,1,CHUWI MiniBook X Laptop Tablet 2 In 1 Intel N1...,618885.94,//www.aliexpress.com/store/1102773765
1,2,MSI Titan 18 Ultra 2024 14th Generation 18-inc...,9355525.54,//www.aliexpress.com/store/1102964053
2,3,FIREBAT A14 Laptop Intel N5095 14.1 Inch 16GB ...,421269.19,//www.aliexpress.com/store/5250057
3,4,ASUS ROG Strix SCAR 18 Gaming Laptop Intel Cor...,5611035.00,//www.aliexpress.com/store/1102248573
4,5,VCHANCE 15.6 Inch 1080p Portable Monitor Built...,300975.91,//www.aliexpress.com/store/1102820440


In [ ]:
# Discount table
# Replace these with the actual column names from your DataFrame
discount_columns = ['Product Name', 'Sales Price', 'Original Price', 'Extra Discount', 'Discount Percentage']

# Create a new DataFrame for the discount table
discount = df[discount_columns].copy()

# Add DiscountID column with unique values
discount['DiscountID'] = range(1, len(discount) + 1)

# Remove Currency Symbols and Commas, then Convert to Float
discount['Sales Price'] = discount['Sales Price'].str.replace('NGN', '').str.replace(',', '').astype(float)
discount['Original Price'] = discount['Original Price'].str.replace('NGN', '').str.replace(',', '').astype(float)

# Extract Discount Percent Value and Coin Percent Value as Integer
discount['Discount Percent Value'] = discount['Discount Percentage'].str.extract('-(\d+)%').astype(float)
discount['Extra Discount Value'] = discount['Extra Discount'].str.extract('(\d+)%').astype(float)

# Before merging, ensure 'Product Name' exists in both DataFrames to avoid key errors.
# Merge with Product table on 'Product Name'.
discount = discount.merge(products, on='Product Name', how='left', suffixes=('', '_prod'))

# Fill NaN values in specific columns with 0.0.
# The inplace=True parameter modifies the original DataFrame.
discount.fillna({
    'Original Price': 0.0,
    'Extra Discount Value': 0.0,
    'Discount Percent Value': 0.0
}, inplace=True)

# Slice and reorder the discount column
discount = discount[['DiscountID', 'ProductID', 'Sales Price', 'Original Price', 'Extra Discount Value', 'Discount Percent Value']]

# Display the DataFrame information to verify the changes.
discount.head()

,DiscountID,ProductID,Sales Price,Original Price,Extra Discount Value,Discount Percent Value
0,1,1,618885.94,1785301.16,2.0,65.0
1,2,2,9355525.54,0.00,2.0,0.0
2,3,3,421269.19,953988.48,2.0,55.0
3,4,4,5611035.00,0.00,0.0,0.0
4,5,5,300975.91,0.00,5.0,0.0


In [ ]:
# Sales Table
sales_columns = ['Product Name', 'Sales Price', 'Items Sold']
sales = df[sales_columns].copy()

# Add SalesID column with unique values
sales['SalesID'] = range(1, len(sales) + 1)

# Remove Currency Symbols and Commas, then Convert to Float
sales['Sales Price'] = sales['Sales Price'].str.replace('NGN', '').str.replace(',', '').astype(float)

# Reorder columns for consistency with the ERD schema and fill missing 'Items Sold' values
sales = sales.merge(products, on='Product Name', how='left')[['SalesID', 'ProductID', 'Sales Price', 'Items Sold']]
sales.fillna({'Items Sold': 0.0}, inplace=True)

sales.head()

,SalesID,ProductID,Sales Price,Items Sold
0,1,1,618885.94,333.0
1,2,2,9355525.54,1.0
2,3,3,421269.19,97.0
3,4,4,5611035.00,1.0
4,5,5,300975.91,8.0


In [ ]:
# Shipping table
# Defining the columns for the shipping table
shipping_columns = ['Product Name', 'Shipping Price', 'Shipping Free Status']
shipping = df[shipping_columns].copy()

# Adding ShippingID column with unique values
shipping['ShippingID'] = range(1, len(shipping) + 1)

# Merging with the products table on 'Product Name' to include ProductID
shipping = shipping.merge(products, on='Product Name', how='left') \
                    [['ShippingID', 'ProductID', 'Shipping Price', 'Shipping Free Status']]

# Applying .fillna() in a separate operation
shipping['Shipping Price'] = shipping['Shipping Price'].fillna(0.0)

# Now, the shipping DataFrame should be intact and ready for inspection
shipping.head()


,ShippingID,ProductID,Shipping Price,Shipping Free Status
0,1,1,0.0,Free shipping
1,2,2,0.0,"+Shipping: NGN120,554.21"
2,3,3,0.0,Free shipping
3,4,4,0.0,"+Shipping: NGN381,105.99"
4,5,5,0.0,Free shipping


In [ ]:
# Save the product and sales tables to new CSV files
products.to_csv('products.csv', index=False)
sales.to_csv('sales.csv', index=False)
discount.to_csv('discount.csv', index=False)
shipping.to_csv('shipping.csv', index=False)

print("Product, Sales, Discount and Shipping tables have been saved successfully.")


Product, Sales, Discount and Shipping tables have been saved successfully.


### Loading Layer

In [ ]:
# Define database connection parameters including the database name
db_params = {
    'username': 'postgres',
    'password': 'password',
    'host': 'localhost',
    'port': '5432',
    'database': 'postgres',  # Database name is directly included here
}

# Define the database connection URL with the database
db_url_with_db = f"postgresql://{db_params['username']}:{db_params['password']}@{db_params['host']}:{db_params['port']}/{db_params['database']}"

# Create the database engine with the database name specified
engine = create_engine(db_url_with_db)

# Define the CSV file paths
shipping_csv = "shipping.csv"
discount_csv = "discount.csv"
products_csv = "products.csv"
sales_csv = "sales.csv"

# Load CSV data into DataFrames
shipping_df = pd.read_csv(shipping_csv)
discount_df = pd.read_csv(discount_csv)
products_df = pd.read_csv(products_csv)
sales_df = pd.read_csv(sales_csv)

# Connect to the PostgreSQL server
with engine.connect() as connection:
    # Create tables and load data
    shipping_df.to_sql('Shipping', connection, index=False, if_exists='replace')
    products_df.to_sql('Product', connection, index=False, if_exists='replace')
    sales_df.to_sql('Sales', connection, index=False, if_exists='replace')
    discount_df.to_sql('Discount', connection, index=False, if_exists='replace')

print("Database, tables, and data loaded successfully.")


Database, tables, and data loaded successfully.
